In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/lcd-luna-16/Augmented Data/aug_Malignant_0_7207.jpeg
/kaggle/input/lcd-luna-16/Augmented Data/aug_Malignant_0_8638.jpeg
/kaggle/input/lcd-luna-16/Augmented Data/aug_Normal_0_6642.jpeg
/kaggle/input/lcd-luna-16/Augmented Data/aug_Malignant_0_9761.jpeg
/kaggle/input/lcd-luna-16/Augmented Data/aug_Benign_0_4262.jpeg
/kaggle/input/lcd-luna-16/Augmented Data/aug_Benign_0_6729.jpeg
/kaggle/input/lcd-luna-16/Augmented Data/aug_Malignant_0_3574.jpeg
/kaggle/input/lcd-luna-16/Augmented Data/aug_Malignant_0_2625.jpeg
/kaggle/input/lcd-luna-16/Augmented Data/aug_Normal_0_6278.jpeg
/kaggle/input/lcd-luna-16/Augmented Data/aug_Benign_0_3520.jpeg
/kaggle/input/lcd-luna-16/Augmented Data/aug_Malignant_0_5827.jpeg
/kaggle/input/lcd-luna-16/Augmented Data/aug_Normal_0_6997.jpeg
/kaggle/input/lcd-luna-16/Augmented Data/aug_Normal_0_2401.jpeg
/kaggle/input/lcd-luna-16/Augmented Data/aug_Malignant_0_2138.jpeg
/kaggle/input/lcd-luna-16/Augmented Data/aug_Benign_0_2103.jpeg
/kaggle/input/lcd-l

In [2]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

2024-05-12 18:11:57.844533: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-12 18:11:57.844636: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-12 18:11:57.959769: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
image_files = [os.path.join(dirname, file) for file in os.listdir(dirname) if file.endswith('.jpeg')]
labels = [0 if 'Benign' in file else 1 for file in image_files]

In [5]:
data = pd.DataFrame({'filename': image_files, 'class': labels})

In [6]:
train_df, test_df = train_test_split(data, test_size=0.2, random_state=42)
val_df, test_df = train_test_split(test_df, test_size=0.5, random_state=42)

In [7]:
# Convert the 'class' column to string type
train_df['class'] = train_df['class'].astype(str)
val_df['class'] = val_df['class'].astype(str)
test_df['class'] = test_df['class'].astype(str)

In [8]:
# Data generators
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

In [9]:
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col='filename',
    y_col='class',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

Found 5829 validated image filenames belonging to 2 classes.


In [10]:
val_generator = val_datagen.flow_from_dataframe(
    dataframe=val_df,
    x_col='filename',
    y_col='class',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

Found 729 validated image filenames belonging to 2 classes.


In [11]:
test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    x_col='filename',
    y_col='class',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=False  # Keep the order of predictions for evaluation
)

Found 729 validated image filenames belonging to 2 classes.


In [12]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [13]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
predictions = Dense(2, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)


In [14]:
for layer in base_model.layers:
    layer.trainable = False


In [15]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [17]:
history = model.fit(
    train_generator,
    epochs=20,
    validation_data=val_generator
)


Epoch 1/20
183/183 ━━━━━━━━━━━━━━━━━━━━ 17s 91ms/step - accuracy: 0.7074 - loss: 0.5276 - val_accuracy: 0.6598 - val_loss: 0.5646
Epoch 2/20
183/183 ━━━━━━━━━━━━━━━━━━━━ 17s 90ms/step - accuracy: 0.6984 - loss: 0.5295 - val_accuracy: 0.7133 - val_loss: 0.5285
Epoch 3/20
183/183 ━━━━━━━━━━━━━━━━━━━━ 17s 92ms/step - accuracy: 0.7065 - loss: 0.5279 - val_accuracy: 0.6790 - val_loss: 0.5352
Epoch 4/20
183/183 ━━━━━━━━━━━━━━━━━━━━ 17s 91ms/step - accuracy: 0.7089 - loss: 0.5233 - val_accuracy: 0.7160 - val_loss: 0.5283
Epoch 5/20
183/183 ━━━━━━━━━━━━━━━━━━━━ 17s 91ms/step - accuracy: 0.7046 - loss: 0.5216 - val_accuracy: 0.7133 - val_loss: 0.5274
Epoch 6/20
183/183 ━━━━━━━━━━━━━━━━━━━━ 17s 91ms/step - accuracy: 0.7053 - loss: 0.5233 - val_accuracy: 0.7078 - val_loss: 0.5244
Epoch 7/20
183/183 ━━━━━━━━━━━━━━━━━━━━ 17s 91ms/step - accuracy: 0.7011 - loss: 0.5189 - val_accuracy: 0.7023 - val_loss: 0.5427
Epoch 8/20
183/183 ━━━━━━━━━━━━━━━━━━━━ 17s 91ms/step - accuracy: 0.6948 - loss: 0.5395 - 

In [18]:
test_loss, test_accuracy = model.evaluate(test_generator)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)


23/23 ━━━━━━━━━━━━━━━━━━━━ 3s 123ms/step - accuracy: 0.7138 - loss: 0.5086
Test Loss: 0.49357980489730835
Test Accuracy: 0.7187928557395935


In [20]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import mean_absolute_error, mean_squared_error, matthews_corrcoef, roc_auc_score

# Make predictions on the test set
y_pred_proba = model.predict(test_generator)
y_pred = np.argmax(y_pred_proba, axis=1)
y_true = test_generator.classes

# Calculate accuracy
accuracy = accuracy_score(y_true, y_pred)

# Calculate precision
precision = precision_score(y_true, y_pred)

# Calculate recall
recall = recall_score(y_true, y_pred)

# Calculate F1 score
f1 = f1_score(y_true, y_pred)

# Calculate mean absolute error
mae = mean_absolute_error(y_true, y_pred)

# Calculate mean squared error
mse = mean_squared_error(y_true, y_pred)

# Calculate Matthews correlation coefficient (MCC)
mcc = matthews_corrcoef(y_true, y_pred)

# Calculate ROC AUC score
roc_auc = roc_auc_score(y_true, y_pred_proba[:, 1])

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("Mean Absolute Error:", mae)
print("Mean Squared Error:", mse)
print("Matthews Correlation Coefficient (MCC):", mcc)
print("ROC AUC Score:", roc_auc)

23/23 ━━━━━━━━━━━━━━━━━━━━ 2s 78ms/step
Accuracy: 0.7187928669410151
Precision: 0.7540983606557377
Recall: 0.8553719008264463
F1 Score: 0.8015488867376573
Mean Absolute Error: 0.2812071330589849
Mean Squared Error: 0.2812071330589849
Matthews Correlation Coefficient (MCC): 0.33339473281228105
ROC AUC Score: 0.7955557429583403


In [ ]:
import matplotlib.pyplot as plt

# Plot training and validation accuracy
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# Plot training and validation loss
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

# Predict probabilities for test set
y_pred_proba = model.predict(test_generator)

# Compute ROC curve and ROC area for each class
fpr, tpr, _ = roc_curve(test_generator.classes, y_pred_proba[:, 1])
roc_auc = auc(fpr, tpr)

# Plot ROC curve
plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc="lower right")
plt.show()

In [ ]:
# Retrieve training and validation MSE from history
train_mse = history.history['loss']
val_mse = history.history['val_loss']

# Plot training and validation MSE
plt.plot(train_mse, label='Training MSE')
plt.plot(val_mse, label='Validation MSE')
plt.title('Training and Validation Mean Squared Error (MSE)')
plt.xlabel('Epoch')
plt.ylabel('MSE')
plt.legend()
plt.show()

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', 'mae'])

# Train the model and store the training history
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator
)

# Retrieve training and validation MAE from history
train_mae = history.history['mae']
val_mae = history.history['val_mae']

# Plot training and validation MAE
plt.plot(train_mae, label='Training MAE')
plt.plot(val_mae, label='Validation MAE')
plt.title('Training and Validation Mean Absolute Error (MAE)')
plt.xlabel('Epoch')
plt.ylabel('MAE')
plt.legend()
plt.show()

In [ ]:
import tensorflow.keras.backend as K
import matplotlib.pyplot as plt

# Define custom R-squared metric
def r_squared(y_true, y_pred):
    SS_res =  K.sum(K.square(y_true - y_pred))
    SS_tot = K.sum(K.square(y_true - K.mean(y_true)))
    return 1 - SS_res/(SS_tot + K.epsilon())

# Compile the model with R-squared metric
model.compile(optimizer='adam', loss='mse', metrics=['mae', r_squared])

# Train the model and store the training history
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator
)

# Retrieve training and validation R-squared from history
train_r_squared = history.history['r_squared']
val_r_squared = history.history['val_r_squared']

# Plot training and validation R-squared
plt.plot(train_r_squared, label='Training R-squared')
plt.plot(val_r_squared, label='Validation R-squared')
plt.title('Training and Validation R-squared Coefficient')
plt.xlabel('Epoch')
plt.ylabel('R-squared')
plt.legend()
plt.show()


In [ ]:
import tensorflow.keras.backend as K
import matplotlib.pyplot as plt

# Define custom R-squared metric
def r_squared(y_true, y_pred):
    SS_res =  K.sum(K.square(y_true - y_pred))
    SS_tot = K.sum(K.square(y_true - K.mean(y_true)))
    return 1 - SS_res/(SS_tot + K.epsilon())

# Compile the model with R-squared metric
model.compile(optimizer='adam', loss='mse', metrics=['mae', r_squared])

# Train the model and store the training history
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator
)

# Retrieve training and validation R-squared from history
train_r_squared = history.history['r_squared']
val_r_squared = history.history['val_r_squared']

# Plot training and validation R-squared
plt.plot(train_r_squared, label='Training R-squared')
plt.plot(val_r_squared, label='Validation R-squared')
plt.title('Training and Validation R-squared Coefficient')
plt.xlabel('Epoch')
plt.ylabel('R-squared')
plt.legend()
plt.show()
